# Visualize your 🤗 Hugging Face data
#### 🛠️ Installation and set-up

In [5]:
import numpy as np
from transformers import RobertaTokenizer
import torch

### 🛫 Data and model preparation
#### 🏷️ Loading a dataset

In [2]:
from datasets import load_dataset
dataset = load_dataset("sst2")

/zhome/94/5/127021/miniconda3/envs/roberta_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset sst2 (/zhome/94/5/127021/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)
100%|██████████| 3/3 [00:00<00:00, 489.91it/s]


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})

For demo sub-sample dataset 

In [4]:
small_data_train = dataset['train'].select(range(dataset['train'].num_rows // 10))
# alternative methods
# dataset["train"].shuffle(seed=42).select([i for i in list(range(100))])
small_data_val = dataset['validation'].select(range(dataset['validation'].num_rows // 10)) # dataset["validation"].shuffle(seed=42).select([i for i in list(range(50))])

### ⚙️ Tokenizing the dataset
In a typical NLP workflow, we must first tokenize our dataset.

Converting the stream of characters in the text into a stream of defined "tokens", which can be anything from a smaller set of characters to words from a vocabulary.

We will use a pretrained model, so we inherit its tokenization scheme.

In [6]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

Downloading: 100%|██████████| 899k/899k [00:00<00:00, 1.42MB/s] 
Downloading: 100%|██████████| 456k/456k [00:00<00:00, 863kB/s] 
Downloading: 100%|██████████| 481/481 [00:00<00:00, 378kB/s]


In [13]:
small_data_train['sentence'][:5]

['hide new secretions from the parental units ',
 'contains no wit , only labored gags ',
 'that loves its characters and communicates something rather beautiful about human nature ',
 'remains utterly satisfied to remain the same throughout ',
 'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ']

In [26]:
test_text = ['hello world',' hello world', 'hello world.','hello world?',' hello world ','hello world .']
test_text2 = ['in store and dog',' in store ','in store ',' in store','in store?','in store.','in store .']

In [28]:
tokenizer(test_text2, truncation = True)

{'input_ids': [[0, 179, 1400, 8, 2335, 2], [0, 11, 1400, 1437, 2], [0, 179, 1400, 1437, 2], [0, 11, 1400, 2], [0, 179, 1400, 116, 2], [0, 179, 1400, 4, 2], [0, 179, 1400, 479, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [19]:
tokenizer(small_data_train['sentence'][:5])#, truncation = True)

{'input_ids': [[0, 37265, 92, 3556, 2485, 31, 5, 20536, 2833, 1437, 2], [0, 10800, 5069, 117, 22094, 2156, 129, 6348, 3995, 821, 8299, 1437, 2], [0, 6025, 6138, 63, 3768, 8, 39906, 402, 1195, 2721, 59, 1050, 2574, 1437, 2], [0, 5593, 5069, 19223, 10028, 7, 1091, 5, 276, 1328, 1437, 2], [0, 261, 5, 2373, 13543, 12, 1116, 12, 627, 12, 1396, 11622, 43848, 5739, 5, 17504, 115, 31120, 1899, 62, 1437, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

We then map the tokenizer over our dataset:

In [3]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True)

dataset = load_dataset("sst2")

small_train_dataset = dataset["train"].shuffle(seed=42).select([i for i in list(range(100))])
small_val_dataset = dataset["validation"].shuffle(seed=42).select([i for i in list(range(50))])

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_val = small_val_dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)
model = RobertaForSequenceClassification.from_pretrained('roberta-base')

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


OSError: [Errno 28] No space left on device: '/zhome/94/5/127021/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5.incomplete'